# Jigsaw - Agile Community Rules Classification
### https://www.kaggle.com/competitions/jigsaw-agile-community-rules

## LLM-Based Content Moderation with LoRA Fine-Tuning
This notebook builds upon our earlier zero-shot inference experiments using a Llama-1B model for Reddit content moderation. Previously, we explored:

Token-based inference using Unsloth + vLLM: https://www.kaggle.com/code/vinothkumarsekar89/jigsaw-acrc-llama-1b-infer-automodel-unsloth-vllm

Logit-based batch inference using vLLM: https://www.kaggle.com/code/vinothkumarsekar89/jigsaw-acrc-llama-1b-batch-infer-logits-vllm

These approaches relied solely on zero-shot prompting without any model fine-tuning.

In this notebook, we fine-tune the Llama-1B model using LoRA (Low-Rank Adaptation) on the provided training dataset to improve classification performance. We then compare the fine-tuned model’s performance against the previous zero-shot baselines using F1-score metrics on a validation set Further... 

Inference using fine-tuned model will be available here: WIP

## Install packages on Kaggle: Add-ons > Install Dependencies 

```bash
pip install pip3-autoremove
pip install torch torchvision torchaudio xformers --index-url https://download.pytorch.org/whl/cu124
pip install unsloth vllm
pip install scikit-learn
```

In [5]:
import kagglehub
import pandas as pd
import os
import glob

# Check if running on Kaggle
if 'KAGGLE_KERNEL_RUN_TYPE' in os.environ:
    # Running on Kaggle
    base_path = "/kaggle/input/jigsaw-agile-community-rules/"
    df_train = pd.read_csv(f"{base_path}train.csv")
    df_test = pd.read_csv(f"{base_path}test.csv")
else:
    # Running locally
    base_path = "./data/final/"
    
    # Find all train files
    train_files = glob.glob(f"{base_path}df_train.csv")
    if train_files:
        train_dfs = [pd.read_csv(file) for file in train_files]
        df_train = pd.concat(train_dfs, ignore_index=True)
        print(f"Concatenated {len(train_files)} train files: {train_files}")
    else:
        raise FileNotFoundError(f"No train files found in {base_path}")
    
    # Find all test files
    test_files = glob.glob(f"{base_path}df_test.csv")
    if test_files:
        test_dfs = [pd.read_csv(file) for file in test_files]
        df_test = pd.concat(test_dfs, ignore_index=True)
        print(f"Concatenated {len(test_files)} test files: {test_files}")
    else:
        raise FileNotFoundError(f"No test files found in {base_path}")

print(f"Train shape: {df_train.shape}")
print(f"Test shape: {df_test.shape}")

print(df_train.columns)
        
req_cols=['subreddit', 'rule', 'positive_example_1', 'negative_example_1', 'positive_example_2',
       'negative_example_2', 'test_comment', 'violates_rule']

df_train=df_train[req_cols]
df_test=df_test[req_cols]

# Normalize "True"/"False" -> "Yes"/"No" and drop anything else
for name, df in [("train", df_train), ("test", df_test)]:
    df["violates_rule"] = (
        df["violates_rule"]
        .astype(str).str.strip()
        .map({"True": "Yes", "False": "No", "Yes": "Yes", "No": "No"})  # normalize
    )
    before = len(df)
    df.dropna(subset=["violates_rule"], inplace=True)  # drop rows with NaN (anything not Yes/No/True/False)
    after = len(df)
    print(f"Dropped {before - after} rows from {name} due to invalid 'violates_rule'")
        
for col in req_cols:
    dropped_rows = df_train[df_train[col].isna()].shape[0]
    print(f"{col}: {dropped_rows} rows would be dropped")
    
df_train = df_train[req_cols].dropna()
df_test = df_test[req_cols].dropna()

print(f"Using path: {base_path}")
print("\n After dropping:")
print(f"Train shape: {df_train.shape}")
print(f"Test shape: {df_test.shape}")
df_train.head(2)

df_train["violates_rule"] = df_train["violates_rule"].astype(str)
df_test["violates_rule"] = df_test["violates_rule"].astype(str)

valid_values = {"Yes", "No"}
df_train = df_train[df_train["violates_rule"].isin(valid_values)]
df_test  = df_test[df_test["violates_rule"].isin(valid_values)]
print("\n After checking True/False:")
print(f"Train shape: {df_train.shape}")
print(f"Test shape: {df_test.shape}")

df_train.to_csv("df_train.csv",index=False)
df_test.to_csv("df_test.csv",index=False)

Concatenated 1 train files: ['./data/final/df_train.csv']
Concatenated 1 test files: ['./data/final/df_test.csv']
Train shape: (24442, 15)
Test shape: (6449, 8)
Index(['subreddit', 'rule', 'positive_example_1', 'negative_example_1',
       'positive_example_2', 'negative_example_2', 'test_comment',
       'violates_rule', 'subreddit_word_count', 'rule_word_count',
       'positive_example_1_word_count', 'negative_example_1_word_count',
       'positive_example_2_word_count', 'negative_example_2_word_count',
       'test_comment_word_count'],
      dtype='object')
Dropped 0 rows from train due to invalid 'violates_rule'
Dropped 0 rows from test due to invalid 'violates_rule'
subreddit: 0 rows would be dropped
rule: 0 rows would be dropped
positive_example_1: 0 rows would be dropped
negative_example_1: 0 rows would be dropped
positive_example_2: 0 rows would be dropped
negative_example_2: 0 rows would be dropped
test_comment: 0 rows would be dropped
violates_rule: 0 rows would be dropped

## Set imports, variable names, parameters

In [6]:
from unsloth import FastLanguageModel
import torch
import os
#os.environ["CUDA_LAUNCH_BLOCKING"] = "0"

dtype = ( None )
load_in_4bit = False
load_in_8bit = False

### List of models
#unsloth/Llama-3.2-3B-Instruct
#unsloth/Qwen3-4B
#unsloth/Mistral-Nemo-Base-2407
# "unsloth/Qwen3-4B-unsloth-bnb-4bit"
# "unsloth/Qwen3-8B-unsloth-bnb-4bit"
# "unsloth/Qwen3-14B-unsloth-bnb-4bit"
# "unsloth/Qwen3-32B-unsloth-bnb-4bit"



######---Parameters to change---#######
#kaggle_model_path="/kaggle/input/model/transformers/1b-instruct/1"
local_model_path="unsloth/Qwen3-4B"
base_model="Qwen3-4B-unsloth"

max_seq_length = 2048
Rank=64
sample_len=int(df_train.shape[0])
max_iter_steps=100
Epochs=2

## To upload to kagglehub (model name & variation version)
model_slug="Qwen3-4B-unsloth-jigsaw-acrc-lora-cml"
variation_slug="01"
###--------------------------------###
train_parameters=f"_lora_fp16_r{Rank}_s{sample_len}_e_{Epochs}_msl{max_seq_length}-"
#train_parameters += '-'.join(sorted([file.split('batch_')[1].split('_')[0] for file in train_files]))
train_parameters +="1to9"
print(train_parameters)

_lora_fp16_r64_s24442_e_2_msl2048-1to9


## Load model using unsloth

In [7]:

if 'KAGGLE_KERNEL_RUN_TYPE' in os.environ:
    model_path=kaggle_model_path
else:
    model_path=local_model_path
        
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name=model_path,
    max_seq_length=max_seq_length,
    dtype=dtype,
    load_in_4bit=load_in_4bit,
    load_in_8bit=load_in_8bit
)

print(model.dtype)


==((====))==  Unsloth 2025.8.6: Fast Qwen3 patching. Transformers: 4.55.2. vLLM: 0.10.0.
   \\   /|    NVIDIA GeForce RTX 4070 Ti SUPER. Num GPUs = 1. Max memory: 15.693 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.7.1+cu128. CUDA: 8.9. CUDA Toolkit: 12.8. Triton: 3.3.1
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.31. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

torch.bfloat16


## Set LoRA Config

In [8]:
model = FastLanguageModel.get_peft_model(
    model,
    r = Rank, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 64,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 123,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

Unsloth 2025.8.6 patched 36 layers with 36 QKV layers, 36 O layers and 36 MLP layers.


## Prepare the dataset from train-data

In [10]:
import pandas as pd
from datasets import Dataset
import kagglehub
import os
import glob
def formatting_prompts_func(examples):
    """
    Format Reddit moderation dataset for ChatML training - matches inference format exactly
    """
    
    texts = []
    
    for i in range(len(examples['subreddit'])):
        # Create system message
        system_msg = f"You are a really experienced moderator for the subreddit /r/{examples['subreddit'][i]}. Your job is to determine if the following reported comment violates the given rule. Answer with only \"Yes\" or \"No\"."
        
        # Create user message with the rule and examples
        user_msg = f"""Rule: {examples['rule'][i]}
Example 1:
{examples['positive_example_1'][i]}
Rule violation: Yes
Example 2:
{examples['negative_example_1'][i]}
Rule violation: No
Example 3:
{examples['positive_example_2'][i]}
Rule violation: Yes
Example 4:
{examples['negative_example_2'][i]}
Rule violation: No
Test sentence:
{examples['test_comment'][i]}"""
        
        # Yes/No format
        response = examples['violates_rule'][i]
        
        # Format as ChatML
        text = f"""<|im_start|>system
{system_msg}<|im_end|>
<|im_start|>user
{user_msg}<|im_end|>
<|im_start|>assistant
{response}<|im_end|>"""
        
        texts.append(text)
    
    return {"text": texts}

# Apply to your datasets
dataset_train = Dataset.from_pandas(df_train)
dataset_train = dataset_train.map(formatting_prompts_func, batched=True)

dataset_test = Dataset.from_pandas(df_test)
dataset_test = dataset_test.map(formatting_prompts_func, batched=True)

Map:   0%|          | 0/24442 [00:00<?, ? examples/s]

Map:   0%|          | 0/6449 [00:00<?, ? examples/s]

In [11]:
# Check dataset sample output
dataset_train['text'][0]

'<|im_start|>system\nYou are a really experienced moderator for the subreddit /r/astronomy. Your job is to determine if the following reported comment violates the given rule. Answer with only "Yes" or "No".<|im_end|>\n<|im_start|>user\nRule: "Help maintain community standards through positive modeling and gentle guidance."\nExample 1:\n"Wow, you\'re still confused about the difference between a planet and a star? Just Google it, dude. Seriously, it\'s basic science."\nRule violation: Yes\nExample 2:\n"It\'s understandable to be a little confused about the distinction between planets and stars. Planets orbit stars and don\'t produce their own light, while stars are giant balls of burning gas that emit light and heat. Happy to explain further if you have more questions!"\nRule violation: No\nExample 3:\n"That\'s the dumbest question I\'ve ever seen on this subreddit. Are you even trying to learn anything?"\nRule violation: Yes\nExample 4:\n"That\'s an interesting question! The reason wh

In [12]:
import mlflow
mlflow.set_tracking_uri("ml_runs")  # Or your remote tracking URI
mlflow.set_experiment("lora-experiments")  # Your experiment name

<Experiment: artifact_location='/home/vino/ML_Projects/Jigsaw-ACRC-Kaggle/ml_runs/765520960065650746', creation_time=1754664463802, experiment_id='765520960065650746', last_update_time=1754664463802, lifecycle_stage='active', name='lora-experiments', tags={}>

## Setup SFT trainer & train the model

In [13]:
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset_train,
    eval_dataset = dataset_test,  # Add test dataset here
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False, # Can make training 5x faster for short sequences.
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        num_train_epochs = Epochs, 
        max_steps = max_iter_steps,
        learning_rate = 5e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 10,
        optim = "adamw_8bit", # "adamw_torch" better for fp16
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 123,
        eval_strategy = "steps", 
        eval_steps = 1000, 
        # Save settings - save after each epoch
        save_strategy = "epoch",
        save_total_limit = 5,  # Keep only last 3 checkpoints to save space
        #load_best_model_at_end = True,
        #metric_for_best_model = "eval_loss",
        output_dir = "outputs",
        report_to = "mlflow", # Use this for WandB etc
    ),
)

Unsloth: Tokenizing ["text"]:   0%|          | 0/24442 [00:00<?, ? examples/s]

Unsloth: Tokenizing ["text"]:   0%|          | 0/6449 [00:00<?, ? examples/s]

[2025-09-12 22:45:04,896] [INFO] [real_accelerator.py:254:get_accelerator] Setting ds_accelerator to cuda (auto detect)


/home/vino/anaconda3/envs/kaggle/compiler_compat/ld: cannot find -laio: No such file or directory
collect2: error: ld returned 1 exit status
/home/vino/anaconda3/envs/kaggle/compiler_compat/ld: cannot find -lcufile: No such file or directory
collect2: error: ld returned 1 exit status


[2025-09-12 22:45:05,381] [INFO] [logging.py:107:log_dist] [Rank -1] [TorchCheckpointEngine] Initialized with serialization = False


In [15]:
formatted_model = base_model.replace(".", "").replace("-", "_")
run_name=formatted_model+train_parameters
with mlflow.start_run(run_name=run_name):
    trainer.train()

## Save LoRA Adaptors

In [ ]:
#save merged 16bit
import os
dir_path = f"./lora/{run_name}"
os.makedirs(dir_path, exist_ok=True)
model.save_pretrained(f"{dir_path}/")  # Local saving
tokenizer.save_pretrained(f"{dir_path}/")

## Save model in merged FP16 format (vllm compatible for inference)

In [ ]:
# #save merged 16bit
# import os
# dir_path = f"./lora/{train_parameter}"
# os.makedirs(dir_path, exist_ok=True)
# model.save_pretrained_merged(dir_path, tokenizer, save_method = "merged_16bit")

## Load model,LORA, Save model in merged FP16 format & convert to GPTQ int4

In [ ]:
# from transformers import AutoModelForCausalLM, AutoTokenizer
# from peft import PeftModel
# from auto_gptq import AutoGPTQForCausalLM,  BaseQuantizeConfig

# # 1. Load base model (consider using unquantized fp16 if available)
# base_model_name = "unsloth/Qwen3-14B-unsloth-bnb-4bit"
# model = AutoModelForCausalLM.from_pretrained(base_model_name, torch_dtype="auto")
# tokenizer = AutoTokenizer.from_pretrained(base_model_name)

# # 2. Load LoRA weights and merge
# lora_dir = "./lora/Qwen3_14B_unsloth_bnb_4bit_lora_fp16_r64_s26899_e_2_msl2048-0-1-2-4-5-7-9"
# model = PeftModel.from_pretrained(model, lora_dir)
# model = model.merge_and_unload()

# # 3. Save merged fp16 model
# save_dir = lora_dir + "_merged"
# model.save_pretrained(save_dir)
# tokenizer.save_pretrained(save_dir)

# # 4. Quantize merged model to GPTQ int4
# gptq_config =  BaseQuantizeConfig(bits=4, group_size=128)
# quantized_model = AutoGPTQForCausalLM.from_pretrained(
#     save_dir,
#     quantization_config=gptq_config
# )
# quantized_model.save_pretrained(lora_dir + "_GPTQ")


## Kagglehub login

In [ ]:
# ## You may need this login if you want to upload model to kagglehub from local machine.
# if 'KAGGLE_KERNEL_RUN_TYPE' in os.environ:
#     pass
# else:
#     kagglehub.login()

## Upload LoRA adaptors/model to kaggle

In [ ]:
# # Replace with path to directory containing model files.
# LOCAL_MODEL_DIR = dir_path
# #LOCAL_MODEL_DIR = "./lora/Qwen3_4B_lora_fp16_r64_s26899_e_3_msl2048-0-1-2-4-5-7-9/"

# MODEL_SLUG = model_slug # Replace with model slug.

# # Learn more about naming model variations at
# # https://www.kaggle.com/docs/models#name-model.
# VARIATION_SLUG = variation_slug # Replace with variation slug.

# kagglehub.model_upload(
#   handle = f"vinothkumarsekar89/{MODEL_SLUG}/transformers/{VARIATION_SLUG}",
#   local_model_dir = LOCAL_MODEL_DIR,
#   version_notes = 'LoRA adapter')